<a href="https://colab.research.google.com/github/nss075/weight-prediction-on-food-choices/blob/main/weight_predict_from_food_choices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow_addons.metrics import RSquare

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
data = pd.read_csv('/content/food_coded.csv')

In [ ]:
data

,GPA,Gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food,comfort_food_reasons,comfort_food_reasons_coded,...,soup,sports,thai_food,tortilla_calories,turkey_calories,type_sports,veggies_day,vitamins,waffle_calories,weight
0,2.4,2,1,430,NaN,315.0,1,none,we dont have comfort,9.0,...,1.0,1.0,1,1165.0,345,car racing,5,1,1315,187
1,3.654,1,1,610,3.0,420.0,2,"chocolate, chips, ice cream","Stress, bored, anger",1.0,...,1.0,1.0,2,725.0,690,Basketball,4,2,900,155
2,3.3,1,1,720,4.0,420.0,2,"frozen yogurt, pizza, fast food","stress, sadness",1.0,...,1.0,2.0,5,1165.0,500,none,5,1,900,I'm not answering this.
3,3.2,1,1,430,3.0,420.0,2,"Pizza, Mac and cheese, ice cream",Boredom,2.0,...,1.0,2.0,5,725.0,690,NaN,3,1,1315,"Not sure, 240"
4,3.5,1,1,720,2.0,420.0,2,"Ice cream, chocolate, chips","Stress, boredom, cravings",1.0,...,1.0,1.0,4,940.0,500,Softball,4,2,760,190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,3.5,1,1,610,4.0,420.0,2,"wine. mac and cheese, pizza, ice cream",boredom and sadness,NaN,...,1.0,1.0,5,940.0,500,Softball,5,1,1315,156
121,3,1,1,265,2.0,315.0,2,Pizza / Wings / Cheesecake,Loneliness / Homesick / Sadness,NaN,...,1.0,NaN,4,940.0,500,basketball,5,2,1315,180
122,3.882,1,1,720,NaN,420.0,1,"rice, potato, seaweed soup",sadness,NaN,...,1.0,2.0,5,580.0,690,none,4,2,1315,120
123,3,2,1,720,4.0,420.0,1,"Mac n Cheese, Lasagna, Pizza","happiness, they are some of my favorite foods",NaN,...,2.0,2.0,1,940.0,500,NaN,3,1,1315,135


In [ ]:
data.isna().sum()

GPA                  2
Gender               0
breakfast            0
calories_chicken     0
calories_day        19
                    ..
type_sports         21
veggies_day          0
vitamins             0
waffle_calories      0
weight               2
Length: 61, dtype: int64

In [ ]:
data[(data.isna().sum()[data.isna().sum() >= 10]).index]

,calories_day,comfort_food_reasons_coded,cuisine,exercise,type_sports
0,NaN,9.0,NaN,1.0,car racing
1,3.0,1.0,1.0,1.0,Basketball
2,4.0,1.0,3.0,2.0,none
3,3.0,2.0,2.0,3.0,NaN
4,2.0,1.0,2.0,1.0,Softball
...,...,...,...,...,...
120,4.0,NaN,1.0,2.0,Softball
121,2.0,NaN,NaN,2.0,basketball
122,NaN,NaN,NaN,2.0,none
123,4.0,NaN,1.0,1.0,NaN


In [ ]:
data = data.drop('type_sports', axis=1)

In [ ]:
numeric_nulls = [column for column in data.columns if data.dtypes[column] != 'object' and data.isna().sum()[column] != 0]

In [ ]:
for column in numeric_nulls:
    data[column] = data[column].fillna(data[column].mean())

In [ ]:
{column: list(data[column].unique()) for column in data.columns if data.isna().sum()[column] > 0}

{'GPA': ['2.4',
  '3.654',
  '3.3',
  '3.2',
  '3.5',
  '2.25',
  '3.8',
  '3.904',
  '3.4',
  '3.6',
  '3.1',
  nan,
  '4',
  '2.2',
  '3.87',
  '3.7',
  '3.9',
  '2.8',
  '3',
  '3.65',
  '3.89',
  '2.9',
  '3.605',
  '3.83',
  '3.292',
  '3.35',
  'Personal ',
  '2.6',
  '3.67',
  '3.73',
  '3.79 bitch',
  '2.71',
  '3.68',
  '3.75',
  '3.92',
  'Unknown',
  '3.77',
  '3.63',
  '3.882'],
 'comfort_food': ['none',
  'chocolate, chips, ice cream',
  'frozen yogurt, pizza, fast food',
  'Pizza, Mac and cheese, ice cream',
  'Ice cream, chocolate, chips ',
  'Candy, brownies and soda.',
  'Chocolate, ice cream, french fries, pretzels',
  'Ice cream, cheeseburgers, chips.',
  'Donuts, ice cream, chips',
  'Mac and cheese, chocolate, and pasta ',
  'Pasta, grandma homemade chocolate cake anything homemade ',
  'chocolate, pasta, soup, chips, popcorn',
  'Cookies, popcorn, and chips',
  'ice cream, cake, chocolate',
  'Pizza, fruit, spaghetti, chicken and Potatoes  ',
  'cookies, donuts, c

In [ ]:
data['GPA'] = data['GPA'].replace('Personal ', np.NaN)
data['GPA'] = data['GPA'].replace('Unknown', np.NaN)
data['GPA'] = data['GPA'].replace('3.79 bitch', '3.79')

data['GPA'] = data['GPA'].astype(np.float)

data['GPA'] = data['GPA'].fillna(data['GPA'].mean())

<ipython-input-13-df704260dbf4>:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['GPA'] = data['GPA'].astype(np.float)


In [ ]:
nonnumeric_nulls = [column for column in data.columns if data.dtypes[column] == 'object' and data.isna().sum()[column] != 0]
nonnumeric_nulls.remove('weight')

data = data.drop(nonnumeric_nulls, axis=1)

In [ ]:
data

,GPA,Gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food_reasons_coded,cook,comfort_food_reasons_coded.1,...,self_perception_weight,soup,sports,thai_food,tortilla_calories,turkey_calories,veggies_day,vitamins,waffle_calories,weight
0,2.400,2,1,430,3.028302,315.0,1,9.000000,2.000000,9,...,3.0,1.0,1.000000,1,1165.0,345,5,1,1315,187
1,3.654,1,1,610,3.000000,420.0,2,1.000000,3.000000,1,...,3.0,1.0,1.000000,2,725.0,690,4,2,900,155
2,3.300,1,1,720,4.000000,420.0,2,1.000000,1.000000,1,...,6.0,1.0,2.000000,5,1165.0,500,5,1,900,I'm not answering this.
3,3.200,1,1,430,3.000000,420.0,2,2.000000,2.000000,2,...,5.0,1.0,2.000000,5,725.0,690,3,1,1315,"Not sure, 240"
4,3.500,1,1,720,2.000000,420.0,2,1.000000,1.000000,1,...,4.0,1.0,1.000000,4,940.0,500,4,2,760,190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,3.500,1,1,610,4.000000,420.0,2,2.698113,3.000000,2,...,4.0,1.0,1.000000,5,940.0,500,5,1,1315,156
121,3.000,1,1,265,2.000000,315.0,2,2.698113,3.000000,3,...,4.0,1.0,1.390244,4,940.0,500,5,2,1315,180
122,3.882,1,1,720,3.028302,420.0,1,2.698113,3.000000,3,...,4.0,1.0,2.000000,5,580.0,690,4,2,1315,120
123,3.000,2,1,720,4.000000,420.0,1,2.698113,3.000000,7,...,2.0,2.0,2.000000,1,940.0,500,3,1,1315,135


In [ ]:
data.isna().sum()

GPA                             0
Gender                          0
breakfast                       0
calories_chicken                0
calories_day                    0
calories_scone                  0
coffee                          0
comfort_food_reasons_coded      0
cook                            0
comfort_food_reasons_coded.1    0
cuisine                         0
diet_current_coded              0
drink                           0
eating_changes_coded            0
eating_changes_coded1           0
eating_out                      0
employment                      0
ethnic_food                     0
exercise                        0
father_education                0
fav_cuisine_coded               0
fav_food                        0
fries                           0
fruit_day                       0
grade_level                     0
greek_food                      0
healthy_feeling                 0
ideal_diet_coded                0
income                          0
indian_food   

In [ ]:
data['weight'].unique()

array(['187', '155', "I'm not answering this. ", 'Not sure, 240', '190',
       '180', '137', '125', '116', '110', '264', '123', '185', '145',
       '170', '135', '165', '175', '195', '105', '160', '167', '115',
       '205', nan, '128', '150', '140', '120', '100', '113', '168', '169',
       '200', '265', '192', '118', '210', '112', '144 lbs', '130', '127',
       '129', '260', '184', '230', '138', '156'], dtype=object)

In [ ]:
data['weight'] = data['weight'].replace('Not sure, 240', '240')
data['weight'] = data['weight'].replace('144 lbs', '144')
data['weight'] = data['weight'].replace("I'm not answering this. ", np.NaN)

data = data.dropna(axis=0).reset_index(drop=True)

In [ ]:
data.isna().sum().sum()

0

In [ ]:
data = data.astype(np.float)

<ipython-input-20-bd93a00e21b0>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = data.astype(np.float)


In [ ]:
data

,GPA,Gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food_reasons_coded,cook,comfort_food_reasons_coded.1,...,self_perception_weight,soup,sports,thai_food,tortilla_calories,turkey_calories,veggies_day,vitamins,waffle_calories,weight
0,2.400,2.0,1.0,430.0,3.028302,315.0,1.0,9.000000,2.000000,9.0,...,3.0,1.0,1.000000,1.0,1165.0,345.0,5.0,1.0,1315.0,187.0
1,3.654,1.0,1.0,610.0,3.000000,420.0,2.0,1.000000,3.000000,1.0,...,3.0,1.0,1.000000,2.0,725.0,690.0,4.0,2.0,900.0,155.0
2,3.200,1.0,1.0,430.0,3.000000,420.0,2.0,2.000000,2.000000,2.0,...,5.0,1.0,2.000000,5.0,725.0,690.0,3.0,1.0,1315.0,240.0
3,3.500,1.0,1.0,720.0,2.000000,420.0,2.0,1.000000,1.000000,1.0,...,4.0,1.0,1.000000,4.0,940.0,500.0,4.0,2.0,760.0,190.0
4,2.250,1.0,1.0,610.0,3.000000,980.0,2.0,4.000000,3.000000,4.0,...,5.0,1.0,2.000000,4.0,940.0,345.0,1.0,2.0,1315.0,190.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,3.500,1.0,1.0,610.0,4.000000,420.0,2.0,2.698113,3.000000,2.0,...,4.0,1.0,1.000000,5.0,940.0,500.0,5.0,1.0,1315.0,156.0
118,3.000,1.0,1.0,265.0,2.000000,315.0,2.0,2.698113,3.000000,3.0,...,4.0,1.0,1.390244,4.0,940.0,500.0,5.0,2.0,1315.0,180.0
119,3.882,1.0,1.0,720.0,3.028302,420.0,1.0,2.698113,3.000000,3.0,...,4.0,1.0,2.000000,5.0,580.0,690.0,4.0,2.0,1315.0,120.0
120,3.000,2.0,1.0,720.0,4.000000,420.0,1.0,2.698113,3.000000,7.0,...,2.0,2.0,2.000000,1.0,940.0,500.0,3.0,1.0,1315.0,135.0


In [ ]:
y = data.loc[:, 'weight']
X = data.drop('weight', axis=1)

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=22)

In [ ]:
X.shape

(122, 48)

In [ ]:
inputs = tf.keras.Input(shape=(48,))
x = tf.keras.layers.Dense(256, activation='relu')(inputs)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer='adam',
    loss='mse'
)


batch_size = 32
epochs = 200

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs
)


Epoch 1/200
3/3 [==============================] - 3s 242ms/step - loss: 24525.5664 - val_loss: 23768.9004
Epoch 2/200
3/3 [==============================] - 0s 95ms/step - loss: 22969.0781 - val_loss: 20669.0352
Epoch 3/200
3/3 [==============================] - 0s 98ms/step - loss: 19194.2031 - val_loss: 13961.5674
Epoch 4/200
3/3 [==============================] - 0s 106ms/step - loss: 11553.6982 - val_loss: 4149.3809
Epoch 5/200
3/3 [==============================] - 0s 78ms/step - loss: 2631.1274 - val_loss: 2532.7622
Epoch 6/200
3/3 [==============================] - 0s 70ms/step - loss: 5542.9429 - val_loss: 2022.9690
Epoch 7/200
3/3 [==============================] - 0s 78ms/step - loss: 2815.1763 - val_loss: 1100.6478
Epoch 8/200
3/3 [==============================] - 0s 108ms/step - loss: 1252.5232 - val_loss: 2270.5481
Epoch 9/200
3/3 [==============================] - 0s 96ms/step - loss: 2014.5573 - val_loss: 1827.6484
Epoch 10/200
3/3 [==============================] - 0s

In [ ]:
fig = px.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'x': "Epoch", 'y': "Loss"},
    title="Loss Over Time"
)

fig.show()


In [ ]:
y_preds = np.squeeze(model.predict(X_test))

2/2 [==============================] - 0s 6ms/step


In [ ]:
rsquare = RSquare()

rsquare.update_state(y_test, y_preds)

In [ ]:
print("R^2 Score:", rsquare.result().numpy())

R^2 Score: -0.18959594
